# Persona Intent Parsing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from persona.preprocess import prepare_json_data, pad_sequences, integer_encode, one_hot_encode, one_hot_encode_target
import random

## Preprocess the data

In [3]:
MAX_SEQ_LEN = 15
input_word_model, output_word_model, pairs = \
    prepare_json_data('input', 'output', './data/persona.intents.json', MAX_SEQ_LEN)

READ 816 sentence pairs
Trimmed to 816 sentence pairs
Counting words...
Counted Words:
input 244
output 53


In [4]:
input_seqs = [pair[0] for pair in pairs]
intent = [pair[1] for pair in pairs]
for i in range(10):
    rando = random.randint(0, len(pairs))
    print(input_seqs[rando],"-->" ,intent[rando])

what process is using the ram --> mem.max.process
is an alarm set --> alarm.list
amount of free mem --> mem.free
what process is using all the memory --> mem.max.process
how much mem is {application} using --> mem.usage.application
what process is using the mem --> mem.max.process
what process is using the mem --> mem.max.process
is it {weekday} --> date
open the folder for {place} --> open.place
play pandora --> pandora.play


#### Pad the input
This process will add the PAD, SOS, EOS, and UNK tokens to sequence

In [5]:
padded_input = pad_sequences(input_seqs, MAX_SEQ_LEN)
print(padded_input[1])

['SOS', 'why', 'did', '{skill}', 'fail', 'EOS', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD', 'PAD']


#### One hot encode

In [6]:
one_hot_input = one_hot_encode(padded_input, input_word_model, MAX_SEQ_LEN)
one_hot_input.shape

(816, 15, 244)

In [7]:
one_hot_output = one_hot_encode([intent], output_word_model, len(intent))[0]
print(one_hot_output.shape)

(816, 53)


## Build and Train the Model

In [8]:
from persona.intent.model import IntentModel

Using TensorFlow backend.


In [ ]:
model = IntentModel("onehot")
model = model(one_hot_input, one_hot_output, input_word_model.n_words, output_word_model.n_words)
model.train(summary=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               190976    
_________________________________________________________________
dense_1 (Dense)              (None, 53)                6837      
Total params: 197,813
Trainable params: 197,813
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
816/816 [==============================] - 1s - loss: 4.3591     
Epoch 2/100
816/816 [==============================] - 0s - loss: 3.1963     
Epoch 3/100
816/816 [==============================] - 0s - loss: 3.0723     
Epoch 4/100
816/816 [==============================] - 0s - loss: 3.1043     
Epoch 5/100
816/816 [==============================] - 0s - loss: 3.0413     
Epoch 6/100
816/816 [==============================] - 0s - loss: 2.9494     
Epoch 7/100
816/816 [==============================] - 0s - loss

816/816 [==============================] - 0s - loss: 0.0988     
Epoch 98/100
816/816 [==============================] - 0s - loss: 0.0707     
Epoch 99/100
816/816 [==============================] - 0s - loss: 0.0751     
Epoch 100/100
816/816 [==============================] - 0s - loss: 0.3616     


In [ ]:
from persona.preprocess import pad_sequence

try:
    while True:
        _input = input("input: ")
        padded_input = [pad_sequence(_input, MAX_SEQ_LEN).split()]
        one_hot = one_hot_encode(padded_input, input_word_model, MAX_SEQ_LEN)
        prediction, confidence = model.decode(one_hot, output_word_model)
        print("intent: ", prediction, confidence)
except KeyboardInterrupt:
    pass

input: hello
intent:  whats.your.name 0.364178
input: what is your name
intent:  whats.your.name 0.795767
input: weather today
intent:  weather.next.hour 0.991494
input: how is the weather today
intent:  weather.next.hour 0.994774
input: what time is it
intent:  cpu.total.usage 0.552368
